# Aula 03 - Pré Processamento

---



Leitura e formato dos dados do Titanic:

In [ ]:
import numpy as np
import pandas as pd

df_dataset = pd.read_csv('titanic.data', sep=',', index_col=None)
# a descricao dos dados pode ser obtida em https://www.kaggle.com/c/titanic/data

df_dataset.shape

Uma primeira impressão sobre os dados e os descritores estatísticos:

In [ ]:
display(df_dataset.head(n=5))
display(df_dataset.describe())

A partir dos dados será que existe uma relação entre a classe que estava viajando e quem sobreviveu? Ou o sexo?

In [ ]:
df_dataset[['Pclass', 'Survived']].groupby(['Pclass'], as_index=False).mean().sort_values(by='Survived', ascending=False)

In [ ]:
df_dataset[['Sex', 'Survived']].groupby(['Sex'], as_index=False).mean().sort_values(by='Survived', ascending=False)

A visualização de dados também pode auxiliar no processo. Verificando a idade:

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

g = sns.FacetGrid(df_dataset, col='Survived')
g.map(plt.hist, 'Age', bins=20)

Analisando graficamente a comparação da classe em que viajavam e se sobreviveram ou não, associado à idade:

In [ ]:
grid = sns.FacetGrid(df_dataset, col='Survived', row='Pclass', height=2.2, aspect=1.6)
grid.map(plt.hist, 'Age', alpha=.5, bins=20)
grid.add_legend()

O processo de análise das variáveis pode continuar, o que é chamado de análise exploratória.
Em seguida, observados quais atributos podem ser interessante, a primeira etapa é remover remover os atributos que julgar desnecessários:

In [ ]:
df_removed = df_dataset.drop(['Name','Cabin', 'PassengerId', 'Ticket'], axis = 1)
df_removed.shape

Em seguida, pode-se verificar quais colunas têm valores ausente e se compensa preenchê-las ou removê-las:

In [ ]:
df_removed.isna().sum()/df_removed.shape[0]

Foi verificado que a idade tem certa importância para sobreviver ou não, portanto seria prejudicial removê-la. A coluna Embarked só poderá ser removida mediante alguns testes. Nesse caso, é importante verificar se é preferível eliminá-la ou se é preferível remover as linhas em que esse valor é ausente. Para preencher a coluna de idade é necessário adotar um valor fixo, ou utilizar apenas um conjunto de dados de treino para estabelecer uma estatística.

In [ ]:
# se o desejo for remover as amostras onde idade ou onde embarcou é vazio, poderia ser utilizado o seguinte comando:
df_dropna = df_removed.dropna(subset = ['Age','Embarked'])
df_dropna.shape

In [ ]:
# se o desejo for de preencher com um valor fixo, poderia ser utilizado o seguinte comando:
values = {'Age': 20, 'Embarked': 'NaN'}
df_filled = df_removed.fillna(value=values)
df_filled.shape

Em seguida, é comum verificar se os valores nulos continuam aparecendo para evitar erros:

In [ ]:
df_filled.isna().sum()/df_filled.shape[0]

Removidas as combinações de dados faltantes, é necessário tratar os dados em termos de transformação numérica e simbólica para que os métodos de aprendizado possam ser aplicados:

In [ ]:
df_filled.Sex[df_filled.Sex == 'female'] = 0
df_filled.Sex[df_filled.Sex == 'male'] = 1

df_dummy = pd.get_dummies(df_filled, columns=['Embarked'])

display(df_dummy.columns)
display(df_dummy.shape)

Em seguida, começa a própria etapa de aprendizagem, com a divisão de dados e finalmente a avaliação do modelo:

In [ ]:
from sklearn.model_selection import train_test_split

X_columns = df_dummy.columns.tolist()
X_columns.remove('Survived')

display(X_columns)

X = df_dummy[X_columns]
y = df_dummy.Survived

display(X.shape)
display(y.shape)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=2019)

### Utilizando os vizinhos próximos como algoritmo:

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(X_train, y_train)

In [ ]:
y_pred = knn.predict(X_test)

In [ ]:
from sklearn import metrics
print("Acurácia:", metrics.accuracy_score(y_test, y_pred))

### Utilizando Naive Bayes e algumas variações para analisar os resultados

In [ ]:
from sklearn.naive_bayes import GaussianNB

model = GaussianNB()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

print("Acurácia:", metrics.accuracy_score(y_test, y_pred))

In [ ]:
from sklearn.naive_bayes import BernoulliNB

model = BernoulliNB()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

print("Acurácia:", metrics.accuracy_score(y_test, y_pred))

## Exercício

Explore diferentes técnicas de pré-processamento (como remover as linhas, em vez de preenchê-las, e vice-versa) e mude os parâmetros tanto dos vizinhos próximos quanto dos métodos baseados no teorema de Bayes, para o objetivo de maximizar o resultado de acurácia.